*Note: This notebook is the slide-oriented version; a [fully literate (wordy) version](https://drive.google.com/file/d/1GU-JXlpCT85dqucPieoJxErsKRZ7xN7L/view?usp=sharing) of the notebook is also available. This notebook was designed to be viewed as slides via the [RISE](https://rise.readthedocs.io/en/stable/) notebook extension!*

# Data Preparation I

Data preparation is a *very* broad and central subject in Data Science.

In today's first lecture we'll cover a few key topics:
1. Data "Unboxing"
2. Structural Data Transformations
3. Type Induction/Coercion
4. String Manipulation

## 1. "Unboxing" Data
Recall some basic tools:
- `du -h`: show the disk size of a file in human-readable form
- `head -c 1024`: show me the first 1024 bytes of a file
- your eyes

### Assessing Structure
1. Look out for header info, metadata, comments, etc.
2. Most files you'll run across fall into one of these categories:
  1. **Record per line**: newline-delimited rows of uniform symbol-delimited data.
    - E.g. CSV and TSV files
    - Also newline-delimited rows of uniform but ad-hoc structured text
  2. **Dictionaries/Objects**: explicit key:value pairs, may be nested! Two common cases:
    - Object-per-line: e.g. newline-delimited rows of JSON, XML, YAML, etc. (JSON in this format is sometimes called [json lines or jsonl](https://jsonlines.org/)).
    - Complex object: the entire dataset is one fully-nested JSON, XML or YAML object
  

  3. **Unions**: a mixture of rows from *k* distinct schemas. Two common cases:
    - *Tagged Unions*: each row has an ID or name identifying its schema. Often the tag is in the first column.
    - *Untagged Unions*: the schema for the row must be classified by its content
  4. **Natural Language (prose)**: intended for human consumption.
  5. **Everything else**: A long tail of file formats! If not readable as text, likely some commercial or open source tool will translate.

### Text formats
Be aware of:
- EBCDIC vs. ASCII. 
- Multibyte character encodings 😟: Unicode, UTF-8 and more. You can search the web for resources on these issues.
You can ignore for this class (and often---but not always!---in life).

### Examples
Without further ado, let's unbox some data!

In [ ]:
!du -h data/*

In [1]:
!head -c 2048 data/jc1.txt

﻿_input,_num,_widgetName,_source,_resultNumber,_pageUrl,game_number,bio1,bio2,bio3,contestant1_name,contestant1_score,contestant2_name,contestant2_score,contestant3_name,contestant3_score
,1,Jeopardy_Winners,Jeopardy_Winners,1,http://www.j-archive.com/showgame.php?game_id=3350,Show #5883 - Wednesday March 24 2010,Derek Honoré an attorney from Inglewood California,Tatiana Walton a graphic designer from Cutler Bay Florida,Regina Robbins an arts teacher from New York New York (whose 1-day cash winnings total $38500),Regina,$19401,Tatiana,$7100,Derek,$11900
,2,Jeopardy_Winners,Jeopardy_Winners,2,http://www.j-archive.com/showgame.php?game_id=4400,Show #6756 - Monday January 20 2014,Jon McGuire a software-development manager from Matthews North Carolina,Blake Perkins an environmental scientist from Baton Rouge Louisiana,Sarah McNitt a study abroad adviser originally from Ann Arbor Michigan (whose 4-day cash winnings total $69199),Sarah,$20199,Blake,$0,Jon,$8380
,3,Jeopardy_Winners,Jeopardy_W

What category of data is the file above? Any observations about the data?

Let's look at another:

In [1]:
!head -c 1024 data/jq2.txt

{"category":"HISTORY","air_date":"2004-12-31","question":"'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'","value":"$200","answer":"Copernicus","round":"Jeopardy!","show_number":"4680"}
{"category":"ESPN's TOP 10 ALL-TIME ATHLETES","air_date":"2004-12-31","question":"'No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves'","value":"$200","answer":"Jim Thorpe","round":"Jeopardy!","show_number":"4680"}
{"category":"EVERYBODY TALKS ABOUT IT...","air_date":"2004-12-31","question":"'The city of Yuma in this state has a record average of 4,055 hours of sunshine each year'","value":"$200","answer":"Arizona","round":"Jeopardy!","show_number":"4680"}
{"category":"THE COMPANY LINE","air_date":"2004-12-31","question":"'In 1963, live on \"The Art Linkletter Show\", this company served its billionth burger'","value":"$200","answer":"McDonald\\'s","round":"Jeopardy!","show_number":"4680"}
{"categ

What do you see this time? Category? Interesting features of the data?

Keep in mind: this is *data visualization*! 

### Richer Visualizations and Low-Code Interaction in Trifacta
- We *could* use pandas and matplotlib/seaborn/plotly/etc. I.e. we could write code.
- Instead, try a purpose-built visual data preparation tool: [Trifacta](cloud.trifacta.com), also known as [Google Cloud Dataprep](https://cloud.google.com/dataprep). A visual environment for "low-code" dataprep that is the commercialization of joint research at Berkeley and Stanford in the [Wrangler](http://vis.stanford.edu/wrangler/) and [Potter's Wheel](http://control.cs.berkeley.edu/abc) projects.

- Can interact with Trifacta in its Web UI, or via the `trifacta` Python library.
- `wf = tf.wrangle` uploads data and returns a "wrangle flow" object from Trifacta. 
- The `wf.open()` method opens the Trifacta UI in a new browser tab.

In [2]:
import trifacta as tf
import pandas as pd

In [3]:
jc1 = tf.wrangle('data/jq2.txt')

In [4]:
jc1.open()

Opening https://tfcso.cloud.trifacta.com/data/95398/461787?minimalView=true


'https://tfcso.cloud.trifacta.com/data/95398/461787?minimalView=true'

What do you see in Trifacta that's additional to `head -c`?

### Let's Look at More Files!

OK, moving on to another file. How would you describe this one?

In [ ]:
import trifacta as tf
import pandas as pd

In [ ]:
!head -c 1024 data/mm.txt

How does that differ from the next file?

In [ ]:
!head -c 1024 data/mmp.txt

And how about this one?

In [ ]:
!head -c 1024 data/mmr.txt

## 2. Structural Transformation: From Relations to Matrices and Back
- Matrix $\rightarrow$ Relational works.
- Relational $\rightarrow$ Matrix sometimes works!
- But how?

To start, let's take our matrix in `mm.txt`, and load it into Trifacta.

In [ ]:
mm = tf.wrangle('data/mm.txt')

- Matrix $\rightarrow$ Relation: UNPIVOT <img src="files/unpivot.png">

In [ ]:
mm.open()

- Relation $\rightarrow$ Matrix: PIVOT <img src="files/pivot.png">
- PIVOT(UNPIVOT) = ??

In [ ]:
mm.open()

### Extra Columns
Let's go back to `mmp.txt`. 
- Matrix or relation? 
- Try doing some PIVOT/UNPIVOT work in Trifacta on this.

In [ ]:
mmp = tf.wrangle('data/mmp.txt')

In [ ]:
mmp.open()

### Duplicate Entries and Aggregation
Now let's take relational version in the `mmr.txt` file, PIVOT into `year`x`month` form.

In [ ]:
mmr = tf.wrangle('data/mmr.txt')

What do we need to do here that we didn't before? What are our choices?

In [ ]:
mmr.open()

### Spreadsheets
- You should try this in your favorite spreadsheet, you may see some differences.
- Beware: there may be no UNPIVOT, or it may be hard to find/use

### PIVOT/UNPIVOT and the Relational Model??!
- In SQL? How about in Relational Algebra?
- Consider how we get "output" column names: $\pi_{c1, c2, c3}(T)$. What's true about the subscripts of $\pi$?
- By contrast, where do output column names come from in PIVOT?
- UNPIVOT: quantifier over column names (variables). This is *second order logic*. Relational langauges are based in first order logic (quantifiers over data).
- So *no* PIVOT/UNPIVOT in "pure" SQL
- BUT: most DBMSs extend SQL to do it. E.g. in Postgres the extension is called [crosstab](https://www.postgresql.org/docs/current/tablefunc.html).

### What about performance and scale
- Have you ever seen a table with 10 million rows? 
- Have you ever seen a table with 10 million columns? 

- Performance
- Usability/Queryability
- Statistics: the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) (and the blessing?!)

## 3. Type Induction and Coercion
To begin let's review "statistical" data types. This is a slight refinement from the terms in DS100:
- *nominal* / *categorical*: types that have no inherent ordering, used as names for categories
- *ordinals*: types that are used to encode order. Typically the integers $1, 2, \ldots$
- *cardinals*: types that are used to express cardinality ("how many"). Typically the integers $0,1,\ldots$. Cardinals are common as the output of statistics (frequencies).
- *numerical* / *measures*: types that capture a numerical value or measurement. Typically a real (floating point) number.

### Data types in the wild
- Some systems (DBMSs) enforce/export types
- Very, very common to work with data that has little or no metadata. Must interpret the data somehow! Type "induction".

### Techniques for Type Induction
- Given: a column $c$ of potentially "dirty" data values
- A set of types H. 
- You need to write an algorithm to choose a type. How does it choose?

- "Hard" Rules: E.g. Occam's razor. 
  - Try types from most- to least-specific. (e.g. boolean, int, float, string)
  - Choose the first one that matches *all* the values.
- Minimum Description Length (MDL): See below
- Classification (i.e. Supervised Learning): You know how this goes.

#### MDL
- Like Occam's razor, but account for the "weight" or "penalty" of encoding exceptions. 
- Say $len(v)$ is the bit-length for encoding of a value $v$ "explicitly". 
- Given a type $T$ with $|T|$ distinct values, the bit-length of encoding a value in that type is $log|T|$.
- Let's say that indicator variable $I_T(v) = 1$ if $v \in T$, and $0$ otherwise. 

$$\mbox{MDL} = \min_{T \in H} \sum_{v \in c}(I_T(v)log(|T|) + (1-I_T(v))len(v))$$

- Example: $\{\mbox{'Joe'}, 2, 12, 4750\}$. Assume the default type is "string", which costs us 8 bits per character. 
- 16-bit integers: 

In [ ]:
c = ['Joe', 'Aditya', '12', '4750']
len('JoeAditya')*8 + 2*16

- All strings: 

In [ ]:
c = ['Joe', 'Aditya', '12', '4750']
sum([len(x)*8 for x in c])

#### In practice
- Many systems *need* hard rules
- Others lean MDLish

### Type Coercion/Casting
- Can be done. You may lose data fidelity (e.g. set to NULL)
- Can be useful. E.g. for *statistically* correct treatment, convert IDs to string rather than integer.
- Also useful for smoothing (e.g. quantization)

## 4. String Manipulation
We covered a lot in DS100 and SQL already.

- Typical transforms include the following (names may vary across systems/DSLs):
  - **Split** a string into separate rows/columns
      - Often by position or delimiter
      - Sometimes via parsing: e.g. counting nested parentheses (e.g. JSON/XML rowsplits)
  - **CountMatches**: Create a new column with the count of matches of a pattern in a string column
  - **Extract**: create a column of substrings derived from another column 
  - **Replace**: a (sub)string with a constant, a "captured group", or any string formula (e.g. lowercase, trim, etc)

- Facility with regular expressions goes a VERY long way.
  - Most languages/tools support regex
- All of the string manipulation can be done 
directly in SQL

## Cleaning The Real Data
The previous few files were the results of wrangling a raw dataset. Let's look at that dataset now! It's a scrape of [rainfall data](https://www.cnrfc.noaa.gov/monthly_precip_2020.php) from the website of the National Oceanic and Atmospheric Administration (NOAA).

In [ ]:
!head -c 1024 data/monthly_precip_full.csv

Messy! You can play in bash or pandas if you like. I'll clean it up in Trifacta, illustrate benefits of tools:
- live visualization 
- transform recommendations (software synthesis) 
- Use CS (HCI, AI, Database languages) to reduce the work in DS!

In [ ]:
mpf = tf.wrangle('data/monthly_precip_full.csv')

In [ ]:
mpf.open()